In [1]:
import numpy as np
import pandas as pd
import pandas.api.types
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
import os
import polars as pl
import random

import lightgbm as lgb

In [2]:
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = True
    SEED = 42
    LR = 1e-3
    WD = 1e-6
    EARLY_STOPPING = False
    NOTEBOOK = True
    
    
if Config.DEBUG:
    n_rows = 10**4
else:
    n_rows = None

if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/isic-2024-challenge"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
    
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"
    LOG_DIR = "../data/logs"
    LOSS_DIR = "../data/losses"

In [4]:
train = pl.read_csv(os.path.join(RAW_DIR, "train-metadata.csv")).to_pandas()

In [7]:
# ターゲットの0と1の割合を計算
target_counts = train['target'].value_counts()
total_counts = len(train)

# 割合を計算
target_ratio = target_counts / total_counts

# 結果を表示
print("ターゲットの0と1の割合と数:")
print(target_counts)
print(target_ratio)

# 不均衡度合いを表示
print("\n不均衡度合い:")
print(f"0の数: {target_counts[0]}, 割合: {target_ratio[0]:.4f}")
print(f"1の数: {target_counts[1]}, 割合: {target_ratio[1]:.4f}")


ターゲットの0と1の割合と数:
target
0    400666
1       393
Name: count, dtype: int64
target
0    0.99902
1    0.00098
Name: count, dtype: float64

不均衡度合い:
0の数: 400666, 割合: 0.9990
1の数: 393, 割合: 0.0010
